In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import classification_report, f1_score

# Load the dataset
df = pd.read_csv('./data/train_tfidf_features.csv')

# Prepare the data
df_feature = df.drop(['label', 'id'], axis=1)
df_target = df['label']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df_feature, df_target, test_size=0.2, random_state=42, stratify=df_target)

# smote = SMOTE(random_state=42)
# X_train, y_train = smote.fit_resample(X_train, y_train)

# Initialize the Bernoulli Naive Bayes model

# I did my own testing and found the hyperparameters that work best for me but idk if can find a better one or not

bnb = BernoulliNB(alpha=1,  binarize=0.1)
rf = RandomForestClassifier(n_estimators=400, random_state=42, bootstrap=False)

# Define the ensemble model using VotingClassifier
ensemble = VotingClassifier(estimators=[('bnb', bnb), ('rf', rf)], voting='soft')  # 'soft' uses predicted probabilities

# Predict on the validation set
ensemble.fit(X_train, y_train)

y_pred = ensemble.predict(X_test)
# Print classification report and macro F1 score
print(classification_report(y_test, y_pred))
print("Macro F1 Score:", f1_score(y_test, y_pred, average='macro'))


In [ ]:
df_test = pd.read_csv('./data/test_tfidf_features.csv')
df_test.drop('id', axis=1, inplace=True)
predictions = ensemble.predict(df_test)
predictions_df = pd.DataFrame(predictions, columns=['label'])
test_id = pd.read_csv('./data/test_tfidf_features.csv')['id']
submission_df = pd.concat([test_id, predictions_df], axis=1)
submission_df.to_csv('submission.csv', index=False)